In [ ]:
from Bio import SeqIO
from pathlib import Path
import pandas as pd

In [ ]:
genomes_to_curate = {"NBC_01310" : ["CP108397.1"],
                     "NBC_01080" : ["CP108663.1"]}

In [ ]:
def remove_shorter_end_cds(query, contigs, input_folder):
    genbank_file = input_folder / f"{query}.gbk"
    records = list(SeqIO.parse(genbank_file, "genbank"))
    
    # Define the threshold for "end of the contig"
    threshold = 1000  # Adjust this value as needed
    
    # Iterate through each record in the GenBank file
    for record in records:
        if record.id in contigs:
            print(f"Processing record: {record.id}")
            # Find CDS features at the end of the contig
            cds_at_end = []
            for feature in record.features:
                if feature.type == "CDS":
                    start = int(feature.location.start)
                    end = int(feature.location.end)
                    if start >= len(record.seq) - threshold or end >= len(record.seq) - threshold:
                        cds_at_end.append(feature)
            
            # Print the CDS features found at the end of the contig
            for cds in cds_at_end:
                print(f"CDS found at the end of the contig: {cds.qualifiers.get('locus_tag', ['unknown'])[0]}")
                print(f"CDS length: {len(cds.location)}")
                print(f"Location: {cds.location}")
                print()
            
            # Find the longest CDS feature
            if cds_at_end:
                longest_cds = max(cds_at_end, key=lambda cds: len(cds.location))
                longest_cds_locus_tag = longest_cds.qualifiers.get('locus_tag', ['unknown'])[0]
                # Remove shorter CDS features and their associated gene features
                new_features = []
                for feature in record.features:
                    if feature.type == "CDS" and feature in cds_at_end and feature != longest_cds:
                        continue
                    if feature.type == "gene":
                        gene_locus_tag = feature.qualifiers.get('locus_tag', ['unknown'])[0]
                        if any(gene_locus_tag == cds.qualifiers.get('locus_tag', ['unknown'])[0] for cds in cds_at_end if cds != longest_cds):
                            continue
                    new_features.append(feature)
                record.features = new_features
                print(f"Longest CDS kept at the end of the contig: {longest_cds_locus_tag}")
                print(f"CDS length: {len(longest_cds.location)}")
                print(f"Location: {longest_cds.location}")
            else:
                print("No CDS found at the end of the contig.")
    
    # Write the modified records to a new GenBank file
    output_file = input_folder / f"{query}_modified.gbk"
    SeqIO.write(records, output_file, "genbank")
    print()
    print(f"Modified GenBank file saved as {output_file}")
    return output_file

In [ ]:
input_folder = Path("../input_files")

# Load the GenBank file
query = "NBC_01310"
contigs = ["CP108397.1"]
df = pd.read_csv("../samples.csv").set_index("genome_id")

for k, v in genomes_to_curate.items():
    outfile = remove_shorter_end_cds(k, v, input_folder)
    df.loc[k, "input_file"] = outfile.name

In [ ]:
df.to_csv("../samples_curated.csv")